<a href="https://colab.research.google.com/github/nlviyuli/SentimenAnalisisLSTM/blob/main/Tahap_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi

import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

!pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia

from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.5 MB/s eta 0:00:00


Menambahkan File Data

In [ ]:
# Path ke file CSV
path = '/content/drive/MyDrive/ulasan_aplikasi_jul_sep_2024 (ok).csv'

# Daftar encoding yang akan dicoba
encodings_to_try = [ 'ISO-8859-1', 'cp1252']

# Periksa apakah DataFrame 'df' telah berhasil dibuat atau tidak.
if 'df' in locals():
    # Tampilkan beberapa baris pertama DataFrame jika berhasil
    print(df.head())
else:
    print("Gagal membaca file dengan semua encoding yang dicoba.")

Gagal membaca file dengan semua encoding yang dicoba.


In [ ]:
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stop words) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Pre-Processing
1. Cleaning

In [ ]:
import pandas as pd
import re
import string

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = re.sub(r'\d+', '', text)  # Menghapus Angka
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus Tanda Baca
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # Menghapus karakter non-ASCII (seperti emoji dan simbol khusus)
    return text

# Path ke file CSV
path = '/content/drive/MyDrive/ulasan_aplikasi_jul_sep_2024 (ok).csv'  # Ganti dengan path file CSV Anda

# Daftar encoding yang akan dicoba
encodings_to_try = ['ISO-8859-1', 'cp1252','utf-8']
df = None  # Initialize df to None

# Coba membaca file dengan beberapa opsi penanganan encoding dan error
for encoding in encodings_to_try:
    try:
        # Coba membaca file dengan encoding tertentu
        df = pd.read_csv(path, sep=';', encoding=encoding)
        print(f"File berhasil dibaca dengan encoding: {encoding}")
        # Jika berhasil dibaca, berhenti mencoba encoding lain
        break
    except UnicodeDecodeError:
        # Jika terjadi UnicodeDecodeError, coba encoding berikutnya
        print(f"Gagal membaca file dengan encoding: {encoding}")
    except pd.errors.ParserError:
        print(f"Gagal membaca file dengan encoding: {encoding} dan delimiter: ';'")
    except Exception as e:
        print(f"Terjadi kesalahan: {e}")

if df is not None:
    # Memeriksa apakah kolom 'riview' ada dalam data
    if 'riview' in df.columns:
        # Terapkan fungsi clean_text ke kolom 'riview'
        df['cleaned_riview'] = df['riview'].apply(clean_text)

        # Menampilkan beberapa hasil cleaning
        print("Hasil cleaning:")
        print(df[['riview', 'cleaned_riview']].head(5))  # Menampilkan 5 baris pertama
    else:
        print("Kolom 'riview' tidak ditemukan dalam data.")
else:
  print("Failed to read data with any encoding")

File berhasil dibaca dengan encoding: ISO-8859-1
Hasil cleaning:
                                              riview  \
0  Sangat buruk , dimana mana orang pesan karna b...   
1  Gk efektif. Diberikan resep obat, cuma bisa te...   
2  Apk sy mengalami error, dimana saya sudah meng...   
3  Sangat sangat tidak rekomendasi apalagi CS nya...   
4  Baru seminggu d update,sudah minta update lagi...   

                                      cleaned_riview  
0  Sangat buruk  dimana mana orang pesan karna bu...  
1  Gk efektif Diberikan resep obat cuma bisa tebu...  
2  Apk sy mengalami error dimana saya sudah mengk...  
3  Sangat sangat tidak rekomendasi apalagi CS nya...  
4  Baru seminggu d updatesudah minta update lagih...  


2. Case Folding

In [ ]:
# Fungsi untuk case folding (mengubah teks menjadi huruf kecil)
def case_folding(text):
    return text.lower()

if 'riview' in df.columns:
    def clean_text(text):
        text = re.sub(r'\d+', '', text) # Menghapus angka
        text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
        text = re.sub(r'[^\x00-\x7F]+', '', text) # Menghapus karakter non-ASCII (seperti emoji dan simbol khusus)
        return text

    df['cleaned_riview'] = df['riview'].apply(clean_text) # Membersihkan kolom 'review' terlebih dahulu

   # Terapkan case folding pada kolom yang sudah dibersihkan
    df['case_folded_riview'] = df['cleaned_riview'].apply(case_folding)

    # Menampilkan beberapa contoh hasil case folding
    print("Hasil case folding setelah cleaning:")
    print(df[['riview', 'cleaned_riview', 'case_folded_riview']].head(5))  # Menampilkan 10 baris pertama

Hasil case folding setelah cleaning:
                                              riview  \
0  Sangat buruk , dimana mana orang pesan karna b...   
1  Gk efektif. Diberikan resep obat, cuma bisa te...   
2  Apk sy mengalami error, dimana saya sudah meng...   
3  Sangat sangat tidak rekomendasi apalagi CS nya...   
4  Baru seminggu d update,sudah minta update lagi...   

                                      cleaned_riview  \
0  Sangat buruk  dimana mana orang pesan karna bu...   
1  Gk efektif Diberikan resep obat cuma bisa tebu...   
2  Apk sy mengalami error dimana saya sudah mengk...   
3  Sangat sangat tidak rekomendasi apalagi CS nya...   
4  Baru seminggu d updatesudah minta update lagih...   

                                  case_folded_riview  
0  sangat buruk  dimana mana orang pesan karna bu...  
1  gk efektif diberikan resep obat cuma bisa tebu...  
2  apk sy mengalami error dimana saya sudah mengk...  
3  sangat sangat tidak rekomendasi apalagi cs nya...  
4  baru seming

In [ ]:
# Menyimpan hasil pembersihan dan case folding ke file CSV baru
df.to_csv('cleaned_and_case_folded_riview.csv', index=False)
# Memberikan pesan bahwa file telah disimpan
print("Hasil pembersihan dan case folding telah disimpan dalam 'cleaned_and_case_folded_riview.csv'.")

Hasil pembersihan dan case folding telah disimpan dalam 'cleaned_and_case_folded_riview.csv'.


3. Tokenizing

In [ ]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize

def tokenizingText(text):
    return word_tokenize(text)

def display_tokenization(text):
    # Tokenisasi teks
    tokens = tokenizingText(text)
    # Menampilkan hasil tokenisasi
    print(f"Original Text: {text}")
    print(f"Tokenized Text: {tokens}\n")

In [ ]:
# Terapkan tokenisasi pada kolom 'case_folded_riview'
df['tokenized_riview'] = df['case_folded_riview'].apply(tokenizingText)
# Tampilkan hasil tokenisasi untuk 5 baris pertama
for i in range(5):
    display_tokenization(df['case_folded_riview'].iloc[i])

Original Text: sangat buruk  dimana mana orang pesan karna butuh cepat  tapi di aplikasi malah memperlama  perhari ini sudah lewat  hari dari yg di tentukan tapi obat pesanan belum juga sampai  chat denga cs tidak ada gunanya  sangat sangat slow respon  status pembayaran dari berhasil berubah menjadi gagal setelah di komplain ke aplikasi nya  halodoc very very bad 
Tokenized Text: ['sangat', 'buruk', 'dimana', 'mana', 'orang', 'pesan', 'karna', 'butuh', 'cepat', 'tapi', 'di', 'aplikasi', 'malah', 'memperlama', 'perhari', 'ini', 'sudah', 'lewat', 'hari', 'dari', 'yg', 'di', 'tentukan', 'tapi', 'obat', 'pesanan', 'belum', 'juga', 'sampai', 'chat', 'denga', 'cs', 'tidak', 'ada', 'gunanya', 'sangat', 'sangat', 'slow', 'respon', 'status', 'pembayaran', 'dari', 'berhasil', 'berubah', 'menjadi', 'gagal', 'setelah', 'di', 'komplain', 'ke', 'aplikasi', 'nya', 'halodoc', 'very', 'very', 'bad']

Original Text: gk efektif diberikan resep obat cuma bisa tebus pakai aplikasi terus pilihan pengiriman

In [ ]:
# Menyimpan DataFrame yang berisi hasil tokenisasi ke dalam file CSV
df.to_csv('/content/tokenized_content.csv', index=False)
# Memberikan pesan untuk memastikan file telah disimpan
print("Hasil tokenisasi telah disimpan ke dalam '/content/tokenized_content.csv'")

Hasil tokenisasi telah disimpan ke dalam '/content/tokenized_content.csv'


4. Stopword

In [ ]:
from nltk.corpus import stopwords

def filteringText(tokens, stopwords_file=None):
    # Daftar stopwords default dari NLTK
    try:
        listStopwords = set(stopwords.words('indonesian'))
    except LookupError:
        import nltk
        nltk.download('stopwords')
        listStopwords = set(stopwords.words('indonesian'))

    # Tambahkan stopwords umum informal (non-negasi)
    additional_stopwords = [
        'yaa', 'yah', 'deh', 'nih', 'dong', 'aja', 'doang', 'tok', 'cuma',
        'cmn', 'kok', 'loh', 'mah', 'pun', 'lah', 'tuh', 'kan', 'kayak',
        'dah', 'ntar', 'trus', 'lagi', 'banget', 'bgt', 'makasih', 'makasi',
        'makas', 'thanks', 'thx', 'pls', 'please', 'btw', 'wkwk', 'wkwkwk',
        'haha', 'hehe', 'hmm', 'hadeh', 'eh', 'sip', 'ok', 'oke', 'yap',
        'sumpah', 'fix', 'gitu', 'gimana', 'gini', 'gituan', 'biasa',
        'biasanya', 'jadiin', 'emang', 'emg', 'ajaib', 'seringan', 'd', 'yg',
        'sy', 'lha', 'very', 'x'
    ]
    listStopwords.update(additional_stopwords)

    # Filtering token
    filtered_tokens = [word for word in tokens if word not in listStopwords]
    return filtered_tokens

In [ ]:
# Terapkan stopword removal untuk setiap baris dalam tokenized riview
df['filtered_riview'] = df['tokenized_riview'].apply(filteringText)

for index, row in df.head(5).iterrows(): #tampilkan 5 baris awal
    print(f"Original Tokenized Text: {row['tokenized_riview']}")
    print(f"Filtered Text (No Stopwords): {row['filtered_riview']}")
    print("\n")

Original Tokenized Text: ['sangat', 'buruk', 'dimana', 'mana', 'orang', 'pesan', 'karna', 'butuh', 'cepat', 'tapi', 'di', 'aplikasi', 'malah', 'memperlama', 'perhari', 'ini', 'sudah', 'lewat', 'hari', 'dari', 'yg', 'di', 'tentukan', 'tapi', 'obat', 'pesanan', 'belum', 'juga', 'sampai', 'chat', 'denga', 'cs', 'tidak', 'ada', 'gunanya', 'sangat', 'sangat', 'slow', 'respon', 'status', 'pembayaran', 'dari', 'berhasil', 'berubah', 'menjadi', 'gagal', 'setelah', 'di', 'komplain', 'ke', 'aplikasi', 'nya', 'halodoc', 'very', 'very', 'bad']
Filtered Text (No Stopwords): ['buruk', 'dimana', 'orang', 'pesan', 'karna', 'butuh', 'cepat', 'aplikasi', 'memperlama', 'perhari', 'tentukan', 'obat', 'pesanan', 'chat', 'denga', 'cs', 'gunanya', 'slow', 'respon', 'status', 'pembayaran', 'berhasil', 'berubah', 'gagal', 'komplain', 'aplikasi', 'nya', 'halodoc', 'bad']


Original Tokenized Text: ['gk', 'efektif', 'diberikan', 'resep', 'obat', 'cuma', 'bisa', 'tebus', 'pakai', 'aplikasi', 'terus', 'pilihan', '

In [ ]:
# Menyimpan hasil stopword removal ke dalam file CSV
df.to_csv('/content/filtered_riview.csv', index=False)
# Memberikan pesan bahwa file telah disimpan
print("Hasil stopword removal telah disimpan ke dalam '/content/filtered_riview.csv'")

Hasil stopword removal telah disimpan ke dalam '/content/filtered_riview.csv'


5. Stemming

In [ ]:
def stemmingText(text):
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    words = text.split()   # Memecah teks menjadi daftar kata
    stemmed_words = [stemmer.stem(word) for word in words]  # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_text = ' '.join(stemmed_words) # Menggabungkan kata-kata yang telah distem
    return stemmed_text

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemmingText(text_list):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()

    # Pastikan input berupa list
    if isinstance(text_list, list):
        stemmed_words = [stemmer.stem(word) for word in text_list]  # Stemming per kata dalam list
        return stemmed_words  # Kembalikan sebagai list (atau bisa di-join jadi string jika perlu)
    else:
        return text_list  # Jika bukan list, kembalikan tanpa perubahan

# Terapkan stemming pada kolom 'filtered_riview'
df['stemmed_riview'] = df['filtered_riview'].apply(stemmingText)
# Tampilkan hasilnya untuk verifikasi
print(df[['stemmed_riview']].head(10))

# Simpan hasilnya ke file CSV
output_file = "stemmed_reviews.csv"  # Nama file keluaran
df.to_csv(output_file, index=False)  # Simpan tanpa menyertakan indeks
print(f"Hasil stemming telah disimpan ke file: {output_file}")

                                      stemmed_riview
0  [buruk, mana, orang, pesan, karna, butuh, cepa...
1  [gk, efektif, resep, obat, tebus, pakai, aplik...
2  [apk, alami, error, mana, konfirmasi, bayar, u...
3  [rekomendasi, cs, nya, beli, obat, konfirmasi,...
4  [minggu, updatesudah, update, lagiheran, gk, a...
5  [kasih, bintang, aplikasi, bagus, mudah, pakai...
6  [aplikasi, nya, mudah, berat, android, versi, ...
7  [aplikasi, bermanfaatmendukungmudah, cari, art...
8  [aplikasi, bagus, manfaat, orang, terima, kasi...
9  [bantu, org, tua, muda, alam, kembang, anak, r...
Hasil stemming telah disimpan ke file: stemmed_reviews.csv


Tahap Pelabelan (Tahap pelabelan dilakukan oleh pakar bahasa)

In [ ]:
import pandas as pd

# Membaca file CSV dengan opsi on_bad_lines='skip' untuk mengabaikan baris yang bermasalah
df = pd.read_csv('/content/drive/MyDrive/stemmed_reviews_pakar_INDO.csv',
                 sep=';',  # Ganti sesuai dengan pemisah yang benar
                 on_bad_lines='skip')

# Menampilkan beberapa baris pertama DataFrame
print(df.head())

                             stemmed_riview;sentimen
0  ['buruk', 'pesan', 'butuh', 'cepat', 'aplikasi...
1  ['efektif', 'resep', 'obat', 'tebus', 'pakai',...
2  ['apk', 'alami', 'error', 'konfirmasi', 'bayar...
3  ['rekomendasi', 'cs', 'beli', 'obat', 'konfirm...
4  ['minggu', 'updatesudah', 'update', 'lagiheran...
